In [45]:
import pandas as pd
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

df = pd.read_csv(fn)
print(df.head())

Saving mobile_legend.csv to mobile_legend (1).csv
User uploaded file "mobile_legend (1).csv" with length 9393 bytes
      Player_ID  TotalMatch  Winrate  HighestRank     Role     Category
0     Tamara_91         754    57.03  grandmaster  Jungler      average
1  Jennifer_757         368    77.83       legend  Jungler          pro
2     Bryan.462         631    46.46       master   Roamer      average
3  Jonathan_622         258    25.96      warrior  Jungler  dark system
4    Nicole.881         372    67.19         epic  Jungler      average


In [83]:
from sklearn.preprocessing import LabelEncoder

# Data contoh role rank
roles = ["Explaner", "Goldlaner", "Jungler", "Midlaner", "Roamer"]
rank = ["warrior", "elite", "master", "grandmaster", "epic", "legend", "mythic"]
# Data kategori skill yang digunakan untuk training
skill_categories = ['pro', 'average', 'dark system']

# Membuat LabelEncoder untuk kategori skill
le_skill = LabelEncoder()
le_skill.fit(skill_categories)  # Latih encoder pada data skill
# Membuat LabelEncoder untuk role rank
le_role = LabelEncoder()
le_role.fit(roles)  # Melatih encoder pada data role rank
le_rank = LabelEncoder()
le_rank.fit(rank)

# Encode data skill ke dalam angka
encoded_skills = le_skill.transform(skill_categories)
print(f"Encoded skill categories: {encoded_skills}")
# Encode data roles rank ke dalam angka
encoded_roles = le_role.transform(roles)
print(f"Encoded roles: {encoded_roles}")
encoded_rank = le_rank.transform(rank)
print(f"Encoded rank: {encoded_rank}")


Encoded roles: [0 1 2 3 4]
Encoded rank: [6 0 4 2 1 3 5]
Encoded skill categories: [2 0 1]


In [84]:
# Menyusun fitur (X) dan target (y)
X = df[['Role', 'Winrate', 'HighestRank', 'TotalMatch']]  # Fitur
y = df['Category']  # Label (target)

# Memisahkan data menjadi data pelatihan dan pengujian
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cek bentuk data pelatihan dan pengujian
print(X_train.shape, X_test.shape)


(160, 4) (40, 4)


In [85]:
from sklearn.ensemble import RandomForestClassifier

# Membuat dan melatih model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prediksi pada data uji
y_pred = model.predict(X_test)

# Evaluasi model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Menampilkan hasil evaluasi
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Accuracy: 0.975
Confusion Matrix:
[[25  0  1]
 [ 0  7  0]
 [ 0  0  7]]
Classification Report:
              precision    recall  f1-score   support

     average       1.00      0.96      0.98        26
 dark system       1.00      1.00      1.00         7
         pro       0.88      1.00      0.93         7

    accuracy                           0.97        40
   macro avg       0.96      0.99      0.97        40
weighted avg       0.98      0.97      0.98        40



In [86]:
def predict_skill_category(role_input, win_rate, rank, total_matches):
    # Mengecek validitas role dan encoding
    if role_input not in le_role.classes_:
        print(f"Role '{role_input}' tidak valid. Pastikan role sudah tepat.")
        return None

    # Encode role menjadi nilai numerik
    role_encoded = le_role.transform([role_input])[0]

    # Mengecek apakah Rank input valid
    if rank not in le_rank.classes_:
        print("Rank tidak valid. Silakan masukkan rank yang benar.")
        return None

    rank_encoded = le_rank.transform([rank])[0]

    # Menyiapkan data input untuk prediksi
    input_data = [[role_encoded, win_rate, rank_encoded, total_matches]]

    # Melakukan prediksi dengan model yang sudah dilatih
    predicted_category_encoded = model.predict(input_data)

    # Validasi label yang diprediksi sebelum inverse_transform
    try:
        predicted_category = le_skill.inverse_transform(predicted_category_encoded)
        return predicted_category[0]
    except ValueError:
        print(f"Label '{predicted_category_encoded[0]}' tidak valid, menggunakan label default 'dark system'.")
        return 'dark system'

# Contoh inputan pengguna
role_input = "Explaner"  # Input Role
win_rate_input = 60  # Input Win Rate
rank_input = "master"  # Input Rank
total_matches_input = 120  # Input Total Matches

# Hasil prediksi kategori skill
predicted_skill = predict_skill_category(role_input, win_rate_input, rank_input, total_matches_input)
if predicted_skill:
    print(f"Prediksi kategori skill pengguna adalah: {predicted_skill}")


Label 'average' tidak valid, menggunakan label default 'dark system'.
Prediksi kategori skill pengguna adalah: dark system


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
